<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания 


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

[ваш текст]

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) и реализуйте полиморфизм с перекрытием и прегегрузкой методов, а также generic классы

<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [ ]:
using System;
using System.Collections.Generic;

class LineItem
{
    private decimal _price;
    private int _quantity;
    private bool _returnAllowed;
    private int _returnedQuantity;

    public string Description { get; set; }

    public decimal Price
    {
        get { return _price; }
        set { _price = value; }
    }

    public int Quantity
    {
        get { return _quantity; }
        set { _quantity = value; }
    }

    public bool ReturnAllowed
    {
        get { return _returnAllowed; }
        set { _returnAllowed = value; }
    }

    public int ReturnedQuantity
    {
        get { return _returnedQuantity; }
        set { _returnedQuantity = value; }
    }

    public LineItem(string description, decimal price, int quantity, bool returnAllowed)
    {
        Description = description;
        _price = price;
        _quantity = quantity;
        ReturnAllowed = returnAllowed;
        _returnedQuantity = 0;
    }

    public virtual decimal GetTotal()
    {
        return _price * (_quantity - _returnedQuantity);
    }

    public virtual bool Return(int returnQuantity)
    {
        if (_returnAllowed && returnQuantity <= (_quantity - _returnedQuantity))
        {
            _returnedQuantity += returnQuantity;
            return true;
        }
        return false;
    }
}
class Invoice<T> where T : LineItem
{
    private string _invoiceNumber;
    private DateTime _issueDate;
    private decimal _totalAmount;
    public List<T> lineItems;

    public string InvoiceNumber
    {
        get { return _invoiceNumber; }
        set { _invoiceNumber = value; }
    }

    public DateTime IssueDate
    {
        get { return _issueDate; }
        set { _issueDate = value; }
    }

    public decimal TotalAmount
    {
        get { return _totalAmount; }
        set { _totalAmount = value; }
    }

    public Invoice(string invoiceNumber, DateTime issueDate)
    {
        _invoiceNumber = invoiceNumber;
        _issueDate = issueDate;
        lineItems = new List<T>();
    }

    public virtual void CalculateTotal()
    {
        _totalAmount = 0;
        foreach (var item in lineItems)
        {
            _totalAmount += item.GetTotal();
        }
    }

    public void AddLine(LineItem lineItem)
    {
        lineItems.Add(lineItem  as T);
        Console.WriteLine($"Добавлена позиция: {lineItem.Description}");
        CalculateTotal();
    }

    // Добавил перегрузку, теперь можем добавлять несколько позиций в фактуру
    public void AddLine(params T[] items)
    {
        foreach (var item in items)
        {
            lineItems.Add(item);
            CalculateTotal();
        }
    }

    public virtual void RemoveLine(T lineItem)
    {
        lineItems.Remove(lineItem);
        CalculateTotal();
    }
}

class GoodsItem : LineItem
{
    public DateTime SupplyDate { get; set; }

    public GoodsItem(string description, decimal price, int quantity, DateTime supplyDate, bool returnAllowed)
        : base(description, price, quantity, returnAllowed)
    {
        SupplyDate = supplyDate;
    }

    public override bool Return(int returnQuantity)
    {
        Console.WriteLine($"Попытка вернуть товар: {Description}");
        return base.Return(returnQuantity);
    }
}

class ServiceItem : LineItem
{
    public DateTime ServiceDate { get; set; }

    public ServiceItem(string description, decimal price, int quantity, DateTime serviceDate, bool returnAllowed)
        : base(description, price, quantity, returnAllowed)
    {
        ServiceDate = serviceDate;
    }

    public override bool Return(int returnQuantity)
    {
        Console.WriteLine($"Попытка вернуть услугу: {Description}");
        return base.Return(returnQuantity);
    }
}

class CombinedInvoice : Invoice<LineItem>
{
    private bool _returnAllowed;
    private int _totalReturnedItems;

    public bool ReturnAllowed
    {
        get { return _returnAllowed; }
        set { _returnAllowed = value; }
    }


    public int TotalReturnedItems
    {
        get { return _totalReturnedItems; }
        private set { _totalReturnedItems = value; }
    }

    public CombinedInvoice(string invoiceNumber, DateTime issueDate, bool returnAllowed)
        : base(invoiceNumber, issueDate)
    {
        _returnAllowed = returnAllowed;
    }

    public void ReturnItem(LineItem lineItem, int returnQuantity)
    {
        if (!_returnAllowed)
        {
            Console.WriteLine("Возврат для данной фактуры запрещен.");
            return;
        }

        if (!lineItem.ReturnAllowed)
        {
            Console.WriteLine($"Возврат для позиции {lineItem.Description} запрещен.");
            return;
        }

        if (lineItem.Return(returnQuantity))
        {
            Console.WriteLine($"Возвращено {returnQuantity} шт. из {lineItem.Description}. Остаток: {lineItem.Quantity - lineItem.ReturnedQuantity}.");
            _totalReturnedItems += returnQuantity;
            CalculateTotal();
            Console.WriteLine($"Общая сумма после возврата: {TotalAmount}.");
        }
        else
        {
            Console.WriteLine($"Невозможно вернуть {returnQuantity} шт. из {lineItem.Description}. Проверьте доступное количество.");
        }
    }
     // Перегрузка оператора + для комбинирования двух фактур
    public static CombinedInvoice operator + (CombinedInvoice inv1, CombinedInvoice inv2)
    {
        // Создаем новую комбинированную фактуру, используя данные первой фактуры
        CombinedInvoice combinedInvoice = new CombinedInvoice(inv1.InvoiceNumber, inv1.IssueDate, inv1.ReturnAllowed);

        // Копируем все позиции из первой фактуры
        foreach (var item in inv1.lineItems)
        {
            combinedInvoice.AddLine(item);
        }

        // Копируем все позиции из второй фактуры
        foreach (var item in inv2.lineItems)
        {
            combinedInvoice.AddLine(item);
        }

        combinedInvoice.CalculateTotal();
        return combinedInvoice;
    }
}

// Пример использования
// CombinedInvoice combinedInvoice = new CombinedInvoice("INV003", DateTime.Now, true);

// GoodsItem item1 = new GoodsItem("Товар 1", 100, 5, DateTime.Now.AddDays(7), true);
// GoodsItem item2 = new GoodsItem("Товар 2", 200, 3, DateTime.Now.AddDays(10), false);
// combinedInvoice.AddLine(item1);

// ServiceItem service1 = new ServiceItem("Услуга 1", 300, 1, DateTime.Now.AddDays(-2), true);
// ServiceItem service2 = new ServiceItem("Услуга 2", 400, 1, DateTime.Now.AddDays(1), false);
// combinedInvoice.AddLine(service1, service2); // Перегрузка

// combinedInvoice.CalculateTotal();
// Console.WriteLine($"Общая сумма фактуры: {combinedInvoice.TotalAmount}");

// combinedInvoice.ReturnItem(item1, 2);

// Console.WriteLine($"Общее количество возвращенных товаров и услуг: {combinedInvoice.TotalReturnedItems}.");
CombinedInvoice combinedInvoice1 = new CombinedInvoice("INV003", DateTime.Now, true);
GoodsItem item1 = new GoodsItem("Товар 1", 100, 5, DateTime.Now.AddDays(7), true);
ServiceItem service1 = new ServiceItem("Услуга 1", 300, 1, DateTime.Now.AddDays(-2), true);
combinedInvoice1.AddLine(item1, service1);

Console.WriteLine($"Общая сумма комбинированной фактуры: {combinedInvoice1.TotalAmount}");
Console.WriteLine($"Количество позиций в комбинированной фактуре: {combinedInvoice1.lineItems.Count}");

CombinedInvoice combinedInvoice2 = new CombinedInvoice("INV004", DateTime.Now, true);
GoodsItem item2 = new GoodsItem("Товар 2", 150, 3, DateTime.Now.AddDays(10), false);
ServiceItem service2 = new ServiceItem("Услуга 2", 400, 1, DateTime.Now.AddDays(1), false);
combinedInvoice2.AddLine(item2, service2);

Console.WriteLine($"Общая сумма комбинированной фактуры: {combinedInvoice2.TotalAmount}");
Console.WriteLine($"Количество позиций в комбинированной фактуре: {combinedInvoice2.lineItems.Count}");

CombinedInvoice combinedInvoice3 = combinedInvoice1 + combinedInvoice2;


Console.WriteLine($"Общая сумма комбинированной фактуры: {combinedInvoice3.TotalAmount}");
Console.WriteLine($"Количество позиций в комбинированной фактуре: {combinedInvoice3.lineItems.Count}");

